In [5]:
import logging
import os

import pandas as pd

from napistu import utils
from napistu.gcs import downloads

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("napistu")

import httpx

NAPISTU_DATA_DIR = os.path.expanduser("~/Desktop/DATA/napistu_data")
NAPISTU_ASSET = "human_consensus_w_distances"


In [6]:
from napistu import indices, consensus, sbml_dfs_utils
from napistu.network import net_create

TESTDATA_PATH = "../lib/napistu-py/src/tests/test_data"

pw_index = indices.PWIndex(os.path.join(TESTDATA_PATH, "pw_index_metabolism.tsv"))
sbml_dfs_dict = consensus.construct_sbml_dfs_dict(pw_index)

sbml_dfs = consensus.construct_consensus_model(sbml_dfs_dict, pw_index)
sbml_dfs_utils.display_post_consensus_checks(sbml_dfs.post_consensus_checks())
napistu_graph = net_create.create_napistu_graph(sbml_dfs)

  0%|          | 0/4 [00:00<?, ?it/s]DEBUG:napistu.consensus:Building consensus identifiers for species


DEBUG:napistu.consensus:Creating lookup table for species
DEBUG:napistu.utils:label is not defined in table_schema; adding a constant (1)
DEBUG:napistu.consensus:Preparing consensus table for species
DEBUG:napistu.consensus:Validating consensus table for species
DEBUG:napistu.consensus:Building consensus identifiers for species
DEBUG:napistu.consensus:Creating lookup table for species
DEBUG:napistu.utils:label is not defined in table_schema; adding a constant (1)
DEBUG:napistu.consensus:Preparing consensus table for species
DEBUG:napistu.consensus:Validating consensus table for species
DEBUG:napistu.consensus:Building consensus identifiers for species
DEBUG:napistu.consensus:Creating lookup table for species
DEBUG:napistu.utils:label is not defined in table_schema; adding a constant (1)
DEBUG:napistu.consensus:Preparing consensus table for species
DEBUG:napistu.consensus:Validating consensus table for species
 75%|███████▌  | 3/4 [00:00<00:00, 19.92it/s]DEBUG:napistu.consensus:Building

'Entity type: species, Check type: source_cooccurrence'

pathway_id,glucose metabolism,glycolysis,ppp,tca
pathway_id,,,,
glucose metabolism,80,55,10,17
glycolysis,55,55,9,10
ppp,10,9,41,8
tca,17,10,8,37


'Entity type: species, Check type: ontology_x_source_cooccurrence'

...

'Entity type: compartments, Check type: source_cooccurrence'

pathway_id,glucose metabolism,glycolysis,ppp,tca
pathway_id,,,,
glucose metabolism,8,4,1,3
glycolysis,4,4,1,1
ppp,1,1,1,1
tca,3,1,1,4


'Entity type: compartments, Check type: ontology_x_source_cooccurrence'

pathway_id,glucose metabolism,glycolysis,ppp,tca
bqb_ontology,,,,
BQB_IS::go,8,4,1,4


DEBUG:napistu.network.net_create:DEBUG: creating compartmentalized species features
INFO:napistu.network.net_create:Organizing all network nodes (compartmentalized species and reactions)
INFO:napistu.network.net_create:Formatting edges as a regulatory graph
INFO:napistu.network.net_create_utils:Processing 434 reaction species using the regulatory hierarchy
INFO:napistu.network.net_create:Adding reversibility and other meta-data from reactions_data
INFO:napistu.network.net_create:Creating reverse reactions for reversible reactions on a directed graph
INFO:napistu.network.net_create:Formatting NapistuGraph output
INFO:napistu.network.ng_core:No isolated vertices found to remove


In [ ]:
entity_type = "reactions"

def entity_data_table_to_dict(sbml_dfs, entity_type: str, table_name: str) -> dict:    
    
    entity_data = sbml_dfs._get_entity_data(entity_type, table_name)
    
    attrs = {}
    for c in entity_data.columns:
        attr_name = table_name + "_" + c
        
        attrs[attr_name] = {
            WEIGHTING_SPEC.TABLE: table_name,
            WEIGHTING_SPEC.VARIABLE: c,
            WEIGHTING_SPEC.TRANSFORMATION: "identity"
        }
    
    return attrs

def entity_data_to_dict(sbml_dfs, entity_type: str, table_names: Optional[list[str]] = None) -> dict:

    entity_data_attr = ENTITIES_TO_ENTITY_DATA[entity_type]
    entity_data = getattr(sbml_dfs, entity_data_attr)

    if len(entity_data) == 0:
        raise ValueError(f"No tables in {entity_data_attr}")

    if table_names is None:
        table_names = entity_data.keys()
    else:
        invalid_tables = set(table_names) - set(entity_data.keys())
        if invalid_tables:
            raise ValueError(f"The following requested tables do not exist in {entity_data_attr}: {invalid_tables}")

    entity_data_dicts = {}
    for table_name in table_names:
        entity_data_dicts[table_name] = entity_data_table_to_dict(sbml_dfs, entity_type, table_name)

    # flatten dict
    cross_table_entity_dict = {k: v for d in entity_data_dicts.values() for k, v in d.items()}

    return cross_table_entity_dict



In [ ]:
# augment edges
napistu_graph.set_graph_attrs({
    "reactions" : entity_data_to_dict(sbml_dfs, SBML_DFS.REACTIONS)
}, mode = "extend")

# add the new attributes
napistu_graph.add_edge_data(sbml_dfs, mode = "extend")